In [1]:
!pip install jsonlines
import jsonlines
import pandas as pd
import plotly.graph_objects as go
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np

In [2]:
rows = []
with jsonlines.open("/content/customer_churn_mini.json") as reader:
    for obj in reader:
        rows.append(obj)

df = pd.DataFrame(rows)
df.head()

,ts,userId,sessionId,page,auth,method,status,level,itemInSession,location,userAgent,lastName,firstName,registration,gender,artist,song,length
0,1538352117000,30,29,NextSong,Logged In,PUT,200,paid,50,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,Freeman,Colin,1.538173e+12,M,Martha Tilston,Rockpools,277.89016
1,1538352180000,9,8,NextSong,Logged In,PUT,200,free,79,"Boston-Cambridge-Newton, MA-NH","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",Long,Micah,1.538332e+12,M,Five Iron Frenzy,Canada,236.09424
2,1538352394000,30,29,NextSong,Logged In,PUT,200,paid,51,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,Freeman,Colin,1.538173e+12,M,Adam Lambert,Time For Miracles,282.82730
3,1538352416000,9,8,NextSong,Logged In,PUT,200,free,80,"Boston-Cambridge-Newton, MA-NH","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",Long,Micah,1.538332e+12,M,Enigma,Knocking On Forbidden Doors,262.71302
4,1538352676000,30,29,NextSong,Logged In,PUT,200,paid,52,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,Freeman,Colin,1.538173e+12,M,Daft Punk,Harder Better Faster Stronger,223.60771


## EDA

In [3]:
# In this section I am trying to understand the provided features and its types and what dose it represent
# Also I am trying to understand the interdependencies between the features

df['ts'] = pd.to_datetime(df['ts'], unit='ms')
df['registration'] = pd.to_datetime(df['registration'], unit='ms')


print(df.shape)
print(df.dtypes)

(286500, 18)
ts               datetime64[ns]
userId                   object
sessionId                 int64
page                     object
auth                     object
method                   object
status                    int64
level                    object
itemInSession             int64
location                 object
userAgent                object
lastName                 object
firstName                object
registration     datetime64[ns]
gender                   object
artist                   object
song                     object
length                  float64
dtype: object


In [4]:
## how many userID & how many sessionID as i concedier them to be the unqiue identifiers
print(len(df['userId'].unique()))
print(len(df['sessionId'].unique()))

226
2354


In [5]:
# Findng the user with most sessions
df.groupby('userId', as_index=False).count().sort_values(by='sessionId', ascending=False).iloc[0:5]
## We conclude that there is users with no userID

,userId,ts,sessionId,page,auth,method,status,level,itemInSession,location,userAgent,lastName,firstName,registration,gender,artist,song,length
160,39,9632,9632,9632,9632,9632,9632,9632,9632,9632,9632,9632,9632,9632,9632,8002,8002,8002
0,,8346,8346,8346,8346,8346,8346,8346,8346,0,0,0,0,0,0,0,0,0
218,92,7230,7230,7230,7230,7230,7230,7230,7230,7230,7230,7230,7230,7230,7230,5945,5945,5945
69,140,6880,6880,6880,6880,6880,6880,6880,6880,6880,6880,6880,6880,6880,6880,5664,5664,5664
138,300011,5732,5732,5732,5732,5732,5732,5732,5732,5732,5732,5732,5732,5732,5732,4619,4619,4619


In [6]:
df_sample = df[df['userId'] == '39']

df_sample.head()

,ts,userId,sessionId,page,auth,method,status,level,itemInSession,location,userAgent,lastName,firstName,registration,gender,artist,song,length
410,2018-10-01 06:35:48,39,267,Home,Logged In,GET,200,free,2,"Los Angeles-Long Beach-Anaheim, CA","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",Campbell,Payton,2018-06-15 01:52:21,F,NaN,NaN,NaN
411,2018-10-01 06:35:52,39,267,NextSong,Logged In,PUT,200,free,3,"Los Angeles-Long Beach-Anaheim, CA","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",Campbell,Payton,2018-06-15 01:52:21,F,HYPOCRISY,Orgy In Blood,200.98567
413,2018-10-01 06:39:12,39,267,NextSong,Logged In,PUT,200,free,4,"Los Angeles-Long Beach-Anaheim, CA","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",Campbell,Payton,2018-06-15 01:52:21,F,The Gathering,Rescue Me,380.23791
420,2018-10-01 06:45:32,39,267,NextSong,Logged In,PUT,200,free,5,"Los Angeles-Long Beach-Anaheim, CA","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",Campbell,Payton,2018-06-15 01:52:21,F,The Black Ghosts,Repetition Kills You,225.77587
422,2018-10-01 06:46:16,39,267,Roll Advert,Logged In,GET,200,free,6,"Los Angeles-Long Beach-Anaheim, CA","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",Campbell,Payton,2018-06-15 01:52:21,F,NaN,NaN,NaN


In [7]:
##The behavior below is showing an active user, but it concerns me that the repeated visits to the "help" page
##at same time it is somehow signals that the help page is actually helpful

df_sample.sort_values(by='ts', inplace=True)
fig = go.Figure(go.Scatter(x=df_sample['ts'], y=df_sample['page'], mode='markers'))

fig.show()

/tmp/ipython-input-928774404.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample.sort_values(by='ts', inplace=True)


In [8]:
##The behavior below is rising a question, what type of subscription is this? is it daily?

fig = go.Figure(go.Scatter(x=df_sample['ts'], y=df_sample['level'], mode='lines'))
fig.show()

In [9]:
print(set(df_sample['auth']))
print(set(df_sample['method']))
print(set(df_sample['status']))

{'Logged In'}
{'PUT', 'GET'}
{200, 307, 404}


In [10]:
## FYI
## 307 Temporary Redirect
## 200 OK
## 404 Not Found

df_sample.sort_values(by='ts', inplace=True)

fig = go.Figure(go.Scatter(x=df_sample['ts'], y=df_sample['status'], mode='markers'))

# Show only specific y-axis points
fig.update_yaxes(
    tickvals=list(set(df_sample['status'])),
    # ticktext=['Low', 'High'] # optional custom labels
)

fig.show()

/tmp/ipython-input-3263973973.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [11]:
df_sample.groupby(['level','sessionId', 'itemInSession']).count()#['itemInSession']

ts  userId  page  auth  method  status  \
level sessionId itemInSession                                           
free  267       2               1       1     1     1       1       1   
                3               1       1     1     1       1       1   
                4               1       1     1     1       1       1   
                5               1       1     1     1       1       1   
                6               1       1     1     1       1       1   
...                            ..     ...   ...   ...     ...     ...   
paid  2459      45              1       1     1     1       1       1   
                46              1       1     1     1       1       1   
                47              1       1     1     1       1       1   
                48              1       1     1     1       1       1   
                49              1       1     1     1       1       1   

                               location  userAgent  lastName  firstName  \
level sessionId itemInSession                                             
free  267       2                     1          1         1          1   
                3                     1          1         1          1   
                4                     1          1         1          1   
                5                     1          1         1          1   
                6                     1          1         1          1   
...                                 ...        ...       ...        ...   
paid  2459      45                    1          1         1          1   
                46                    1          1         1          1   
                47                    1          1         1          1   
                48                    1          1         1          1   
                49                    1          1         1          1   

                               registration  gender  artist  song  length  
level sessionId itemInSession                                              
free  267       2                         1       1       0     0       0  
                3                         1       1       1     1       1  
                4                         1       1       1     1       1  
                5                         1       1       1     1       1  
                6                         1       1       0     0       0  
...                                     ...     ...     ...   ...     ...  
paid  2459      45                        1       1       1     1       1  
                46                        1       1       1     1       1  
                47                        1       1       1     1       1  
                48                        1       1       1     1       1  
                49                        1       1       1     1       1  

[9632 rows x 15 columns]

In [12]:
df_sample_treemap = df_sample.groupby([ 'level', 'sessionId', 'itemInSession'], as_index=False).count().sort_values(by='sessionId', ascending=False)
df_sample_treemap

,level,sessionId,itemInSession,ts,userId,page,auth,method,status,location,userAgent,lastName,firstName,registration,gender,artist,song,length
9610,paid,2459,28,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
9616,paid,2459,34,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
9615,paid,2459,33,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
9614,paid,2459,32,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
9613,paid,2459,31,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,free,267,81,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
28,free,267,33,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
29,free,267,34,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
30,free,267,35,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [13]:
## This chart is showing me that every sessionID have multiable itemSessions, and these itemSessions starts from 0 fo each sessionID

import plotly.express as px


fig = px.treemap(df_sample_treemap, path=[px.Constant("df['userId'] == '39'"), 'level', 'sessionId', 'itemInSession'], values='ts')
fig.update_traces(root_color="lightgrey")
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

In [14]:
df_sample['registration'].unique() # = pd.to_datetime(df_sample['registration'], unit='ms')
#df_sample#.head()

<DatetimeArray>
['2018-06-15 01:52:21']
Length: 1, dtype: datetime64[ns]

In [15]:
sorted(df_sample['length'].unique())[-10:]

[np.float64(972.06812),
 np.float64(978.442),
 np.float64(979.56526),
 np.float64(995.13424),
 np.float64(1058.89914),
 np.float64(1121.25342),
 np.float64(1242.46159),
 np.float64(1933.11302),
 np.float64(2520.99873),
 np.float64(3024.66567)]

In [16]:
## FYI the shape of the shataset (286500, 18)
## so the comlumns [location  userAgent lastName firstName registration gender] the null values representing 2%
## and the columns [artist song length] the null values representing 20%
## I am leaning toward removing the [artist song length] features, because I already seeing value with the "page"
## and removing the the rows where the user ID is "" because this user is the cause of the problem with the other features
print(df.isnull().sum())

ts                   0
userId               0
sessionId            0
page                 0
auth                 0
method               0
status               0
level                0
itemInSession        0
location          8346
userAgent         8346
lastName          8346
firstName         8346
registration      8346
gender            8346
artist           58392
song             58392
length           58392
dtype: int64


In [17]:
df[df['gender'].isna()]['userId'].unique()

array([''], dtype=object)

In [18]:
df[df['userId']==''].head()#This part is covering the null values in the [location  userAgent lastName firstName registration gender]

,ts,userId,sessionId,page,auth,method,status,level,itemInSession,location,userAgent,lastName,firstName,registration,gender,artist,song,length
57,2018-10-01 01:02:25,,8,Home,Logged Out,GET,200,free,100,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
58,2018-10-01 01:03:27,,8,Help,Logged Out,GET,200,free,101,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
59,2018-10-01 01:04:01,,8,Home,Logged Out,GET,200,free,102,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
60,2018-10-01 01:04:02,,8,Login,Logged Out,PUT,307,free,103,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
76,2018-10-01 01:17:58,,240,Home,Logged Out,GET,200,free,2,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN


In [19]:
df[df['artist'].isna()].head()#This part is covering the nulls in the columns [artist song length]
#which is the cases where the ['page'] is somehwere not for the music

,ts,userId,sessionId,page,auth,method,status,level,itemInSession,location,userAgent,lastName,firstName,registration,gender,artist,song,length
8,2018-10-01 00:15:05,30,29,Add to Playlist,Logged In,PUT,200,paid,54,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,Freeman,Colin,2018-09-28 22:22:42,M,NaN,NaN,NaN
11,2018-10-01 00:19:10,9,8,Roll Advert,Logged In,GET,200,free,84,"Boston-Cambridge-Newton, MA-NH","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",Long,Micah,2018-09-30 18:20:30,M,NaN,NaN,NaN
14,2018-10-01 00:22:56,9,8,Thumbs Up,Logged In,PUT,307,free,86,"Boston-Cambridge-Newton, MA-NH","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",Long,Micah,2018-09-30 18:20:30,M,NaN,NaN,NaN
35,2018-10-01 00:45:49,54,53,Downgrade,Logged In,GET,200,paid,4,"Spokane-Spokane Valley, WA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:32.0) G...,Warren,Alexi,2018-07-25 01:37:42,F,NaN,NaN,NaN
44,2018-10-01 00:54:15,54,53,Thumbs Up,Logged In,PUT,307,paid,7,"Spokane-Spokane Valley, WA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:32.0) G...,Warren,Alexi,2018-07-25 01:37:42,F,NaN,NaN,NaN


## Data Cleaning & Features Engineering

In [20]:
df_clean = df[df['userId']!=''].drop(columns=['artist','song','length','lastName','firstName'] )
print(df_clean.isnull().sum())

ts               0
userId           0
sessionId        0
page             0
auth             0
method           0
status           0
level            0
itemInSession    0
location         0
userAgent        0
registration     0
gender           0
dtype: int64


In [21]:
df_clean.head()

,ts,userId,sessionId,page,auth,method,status,level,itemInSession,location,userAgent,registration,gender
0,2018-10-01 00:01:57,30,29,NextSong,Logged In,PUT,200,paid,50,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,2018-09-28 22:22:42,M
1,2018-10-01 00:03:00,9,8,NextSong,Logged In,PUT,200,free,79,"Boston-Cambridge-Newton, MA-NH","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",2018-09-30 18:20:30,M
2,2018-10-01 00:06:34,30,29,NextSong,Logged In,PUT,200,paid,51,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,2018-09-28 22:22:42,M
3,2018-10-01 00:06:56,9,8,NextSong,Logged In,PUT,200,free,80,"Boston-Cambridge-Newton, MA-NH","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",2018-09-30 18:20:30,M
4,2018-10-01 00:11:16,30,29,NextSong,Logged In,PUT,200,paid,52,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,2018-09-28 22:22:42,M


In [28]:
temp_ = df_clean
temp_['date']= temp_['ts'].dt.date

x = temp_.groupby(['userId','sessionId']).count()

#df_clean = 278154 rows
#df_clean.groupby(['userId','sessionId', 'itemInSession', 'level']) = 278154 rows
#df_clean.groupby(['userId','sessionId', 'itemInSession']) = 278154 rows
#df_clean.groupby(['userId','sessionId', 'level']) = 3386 rows
#df_clean.groupby(['userId','sessionId']) = 3176 rows
# I conclude that each 'itemInSession' will be in certain subscribtion level
# but each 'sessionId' can have different subscribtion level

#temp_.groupby(['userId','sessionId','date']) = 3817
#temp_.groupby(['userId','sessionId']) = 3176
# I assume each sessionID is in single day except the session that takes overnight which will lead to single session with in two days

x

ts  page  auth  method  status  level  itemInSession  \
userId sessionId                                                          
10     9           70    70    70      70      70     70             70   
       595        449   449   449     449     449    449            449   
       1047        25    25    25      25      25     25             25   
       1414        78    78    78      78      78     78             78   
       1592        78    78    78      78      78     78             78   
...               ...   ...   ...     ...     ...    ...            ...   
99     1683        54    54    54      54      54     54             54   
       2011        24    24    24      24      24     24             24   
       2062        88    88    88      88      88     88             88   
       2201       141   141   141     141     141    141            141   
       2274       133   133   133     133     133    133            133   

                  location  userAgent  registration  gender  date  
userId sessionId                                                   
10     9                70         70            70      70    70  
       595             449        449           449     449   449  
       1047             25         25            25      25    25  
       1414             78         78            78      78    78  
       1592             78         78            78      78    78  
...                    ...        ...           ...     ...   ...  
99     1683             54         54            54      54    54  
       2011             24         24            24      24    24  
       2062             88         88            88      88    88  
       2201            141        141           141     141   141  
       2274            133        133           133     133   133  

[3176 rows x 12 columns]

In [32]:
from datetime import datetime


df_ = df_clean.sort_values(by=['ts', 'userId','sessionId', 'itemInSession'])
df_.drop_duplicates(inplace=True)
df_['date'] =  df_['ts'].dt.date

label = []
different_days = []

df_label = pd.DataFrame(columns=list(df_.columns)+['chrun', 'days_since_last_session'])

for id in df_['userId'].unique() :
  # print(f'Start id {id} Columns {df_label.columns}')
  temp = df_[df_['userId']==id].copy()
  # print(f'Start temp Columns {temp.columns}')

  label = []
  label.append(0)

  different_days = []
  different_days.append(0)

  # print(temp.head())
  for i in range(1,len(temp)):
    current_level = temp.iloc[i]['level']
    previous_level = temp.iloc[(i-1)]['level']
    if (previous_level == 'paid') & (current_level == 'free') :
      label.append(1)
    else :
      label.append(0)

    current_date = temp.iloc[i]['date']
    previous_date = temp.iloc[(i-1)]['date']
    diff = current_date - previous_date

    different_days.append(diff.days)

  temp['chrun'] = label
  temp['days_since_last_session'] = different_days
  #
  df_label = pd.concat([df_label, temp], ignore_index=True)
  # print(f'Finish id {id} Columns {df_label.columns}')
    # break
  # break




/tmp/ipython-input-1185253815.py:41: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



In [35]:
# df_label[df_label['chrun']==1]
# df_label[(df_label['userId']=='30') & (df_label['sessionId']==532)]
sum(df_label['days_since_last_session'])#.iloc[250:270]

9339

In [71]:
# this is showing that there are vey few flags for the chrun flags indicating that among the session there are
# few incidens where the user changes plans from paied to free in comparasom with the incides who staied in the same plan
fig = go.Figure(go.Pie(labels=['staied in same plan','changed to free'], values=[(df_label['chrun']==0).sum(), (df_label['chrun']==1).sum()]))
fig.show()

In [45]:
x = df_label.groupby(['days_since_last_session'], as_index=False).count()


fig = go.Figure(go.Bar(x=x['days_since_last_session'], y=x['sessionId']))
fig.show()

In [50]:
#mostly the users use the app everyday and only few are starting to be absent

zero_days = sum(x[x['days_since_last_session'] == 0 ]['sessionId'])
absent_days = sum(x[x['days_since_last_session'] != 0 ]['sessionId'])

fig = go.Figure(go.Pie(labels=['active','absent'], values=[zero_days, absent_days]))
fig.show()

In [51]:
# Extract features
df_label['year'] = df_label['ts'].dt.year
df_label['month'] = df_label['ts'].dt.month
df_label['day'] = df_label['ts'].dt.day
df_label['hour'] = df_label['ts'].dt.hour
df_label['minute'] = df_label['ts'].dt.minute
df_label['second'] = df_label['ts'].dt.second

# Optionally, drop the original datetime column if not needed
df_label = df_label.drop(columns=['ts'])
df_label

,userId,sessionId,page,auth,method,status,level,itemInSession,location,userAgent,...,gender,date,chrun,days_since_last_session,year,month,day,hour,minute,second
0,30,29,NextSong,Logged In,PUT,200,paid,50,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,...,M,2018-10-01,0,0,2018,10,1,0,1,57
1,30,29,NextSong,Logged In,PUT,200,paid,51,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,...,M,2018-10-01,0,0,2018,10,1,0,6,34
2,30,29,NextSong,Logged In,PUT,200,paid,52,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,...,M,2018-10-01,0,0,2018,10,1,0,11,16
3,30,29,NextSong,Logged In,PUT,200,paid,53,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,...,M,2018-10-01,0,0,2018,10,1,0,14,59
4,30,29,Add to Playlist,Logged In,PUT,200,paid,54,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,...,M,2018-10-01,0,0,2018,10,1,0,15,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278149,200012,400,NextSong,Logged In,PUT,200,free,49,"Portland-Vancouver-Hillsboro, OR-WA","""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",...,M,2018-11-30,0,0,2018,11,30,22,18,1
278150,200012,400,Roll Advert,Logged In,GET,200,free,50,"Portland-Vancouver-Hillsboro, OR-WA","""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",...,M,2018-11-30,0,0,2018,11,30,22,18,1
278151,200012,400,NextSong,Logged In,PUT,200,free,51,"Portland-Vancouver-Hillsboro, OR-WA","""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",...,M,2018-11-30,0,0,2018,11,30,22,21,38
278152,200012,400,NextSong,Logged In,PUT,200,free,52,"Portland-Vancouver-Hillsboro, OR-WA","""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",...,M,2018-11-30,0,0,2018,11,30,22,24,52


In [52]:
# Extract features
df_label['reg_year'] = df_label['registration'].dt.year
df_label['reg_month'] = df_label['registration'].dt.month
df_label['reg_day'] = df_label['registration'].dt.day
df_label['reg_hour'] = df_label['registration'].dt.hour
df_label['reg_minute'] = df_label['registration'].dt.minute
df_label['reg_second'] = df_label['registration'].dt.second

# Optionally, drop the original datetime column if not needed
df_label = df_label.drop(columns=['registration'])
df_label

,userId,sessionId,page,auth,method,status,level,itemInSession,location,userAgent,...,day,hour,minute,second,reg_year,reg_month,reg_day,reg_hour,reg_minute,reg_second
0,30,29,NextSong,Logged In,PUT,200,paid,50,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,...,1,0,1,57,2018,9,28,22,22,42
1,30,29,NextSong,Logged In,PUT,200,paid,51,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,...,1,0,6,34,2018,9,28,22,22,42
2,30,29,NextSong,Logged In,PUT,200,paid,52,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,...,1,0,11,16,2018,9,28,22,22,42
3,30,29,NextSong,Logged In,PUT,200,paid,53,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,...,1,0,14,59,2018,9,28,22,22,42
4,30,29,Add to Playlist,Logged In,PUT,200,paid,54,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,...,1,0,15,5,2018,9,28,22,22,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278149,200012,400,NextSong,Logged In,PUT,200,free,49,"Portland-Vancouver-Hillsboro, OR-WA","""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",...,30,22,18,1,2018,9,26,5,20,56
278150,200012,400,Roll Advert,Logged In,GET,200,free,50,"Portland-Vancouver-Hillsboro, OR-WA","""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",...,30,22,18,1,2018,9,26,5,20,56
278151,200012,400,NextSong,Logged In,PUT,200,free,51,"Portland-Vancouver-Hillsboro, OR-WA","""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",...,30,22,21,38,2018,9,26,5,20,56
278152,200012,400,NextSong,Logged In,PUT,200,free,52,"Portland-Vancouver-Hillsboro, OR-WA","""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",...,30,22,24,52,2018,9,26,5,20,56


In [53]:
cols_to_convert = ['userId', 'sessionId', 'status', 'itemInSession', 'chrun']  # list of object columns

df_label[cols_to_convert] = df_label[cols_to_convert].astype(int)

In [54]:
print(df_label.dtypes)

userId                      int64
sessionId                   int64
page                       object
auth                       object
method                     object
status                      int64
level                      object
itemInSession               int64
location                   object
userAgent                  object
gender                     object
date                       object
chrun                       int64
days_since_last_session    object
year                        int32
month                       int32
day                         int32
hour                        int32
minute                      int32
second                      int32
reg_year                    int32
reg_month                   int32
reg_day                     int32
reg_hour                    int32
reg_minute                  int32
reg_second                  int32
dtype: object


In [55]:
from sklearn.preprocessing import OneHotEncoder
cat_cols = ['level', 'page', 'method', 'auth', 'gender']
encoder = OneHotEncoder(sparse_output=False)
encoded = encoder.fit_transform(df_label[cat_cols])
encoded = encoded.astype(int)

encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(cat_cols))

df_model = pd.concat([df_label, encoded_df], axis=1)
df_model = df_model.drop(columns=cat_cols)

## Handling Unbalance in the dataframe

In [93]:
X_minority = df_model[df_model['chrun'] == 1]
X_majority = df_model[df_model['chrun'] == 0]

# Duplicate minority events
X_minority_upsampled = X_minority.sample(len(X_majority), replace=True, random_state=42)

df_balanced = pd.concat([X_majority, X_minority_upsampled])

In [94]:
df_balanced.sort_values(by=['year','month','day','hour','minute','second'], inplace=True)

In [95]:
fig = go.Figure(go.Pie(labels=['staied in same plan','changed to free'], values=[(df_balanced['chrun']==0).sum(), (df_balanced['chrun']==1).sum()]))
fig.show()

In [72]:
X_minority = df_balanced[df_balanced['days_since_last_session'] != 0 ]
X_majority = df_balanced[df_balanced['days_since_last_session'] == 0 ]

# Duplicate minority events
X_minority_upsampled = X_minority.sample(len(X_majority), replace=True, random_state=42)

df_balanced = pd.concat([X_majority, X_minority_upsampled])

In [73]:
#mostly the users use the app everyday and only few are starting to be absent

zero_days = sum(df_balanced[df_balanced['days_since_last_session'] == 0 ]['sessionId'])
absent_days = sum(df_balanced[df_balanced['days_since_last_session'] != 0 ]['sessionId'])

fig = go.Figure(go.Pie(labels=['active','absent'], values=[zero_days, absent_days]))
fig.show()

In [83]:
df_balanced['days_since_last_session'] = df_balanced['days_since_last_session'].astype(int)
df_balanced = df_balanced.drop(columns='date')

In [84]:
split_index = int(df_balanced.shape[0] * 0.7)

train = df_balanced.iloc[:split_index]
test  = df_balanced.iloc[split_index:]

X_train = train.drop(columns=['chrun', 'location', 'userAgent'])  # returns a new DataFrame
y_train = train['chrun']

X_test = test.drop(columns=['chrun', 'location', 'userAgent'])  # returns a new DataFrame
y_test = test['chrun']

In [85]:
print(X_train.dtypes)

userId                            int64
sessionId                         int64
status                            int64
itemInSession                     int64
days_since_last_session           int64
year                              int32
month                             int32
day                               int32
hour                              int32
minute                            int32
second                            int32
reg_year                          int32
reg_month                         int32
reg_day                           int32
reg_hour                          int32
reg_minute                        int32
reg_second                        int32
level_free                        int64
level_paid                        int64
page_About                        int64
page_Add Friend                   int64
page_Add to Playlist              int64
page_Cancel                       int64
page_Cancellation Confirmation    int64
page_Downgrade                    int64


## Modeling

In [86]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(class_weight='balanced', max_iter=1000)

In [87]:
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.92      0.96    232440
           1       0.00      0.00      0.00         0

    accuracy                           0.92    232440
   macro avg       0.50      0.46      0.48    232440
weighted avg       1.00      0.92      0.96    232440



/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.12/dist-packages/sklearn/

## The large dataset

In [88]:
rows = []
with jsonlines.open("/content/customer_churn.json") as reader:
    for obj in reader:
        rows.append(obj)

df = pd.DataFrame(rows)
df.head()

,ts,userId,sessionId,page,auth,method,status,level,itemInSession,location,userAgent,lastName,firstName,registration,gender,artist,song,length
0,1538352011000,293,292,NextSong,Logged In,PUT,200,free,20,"Corpus Christi, TX","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",Morales,Joseph,1.532064e+12,M,Martin Orford,Grand Designs,597.55057
1,1538352025000,98,97,NextSong,Logged In,PUT,200,free,74,"Houston-The Woodlands-Sugar Land, TX","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",Larson,Sawyer,1.538070e+12,M,John Brown's Body,Bulls,380.21179
2,1538352118000,179,178,NextSong,Logged In,PUT,200,paid,184,"Orlando-Kissimmee-Sanford, FL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",Santiago,Maverick,1.535953e+12,M,Afroman,Because I Got High,202.37016
3,1538352119000,179,178,Logout,Logged In,PUT,307,paid,185,"Orlando-Kissimmee-Sanford, FL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",Santiago,Maverick,1.535953e+12,M,NaN,NaN,NaN
4,1538352124000,246,245,NextSong,Logged In,PUT,200,paid,22,"Mobile, AL",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,Campos,Gianna,1.535931e+12,F,Lily Allen,Smile (Radio Edit),194.53342


In [89]:
# In this section I am trying to understand the provided features and its types and what dose it represent
# Also I am trying to understand the interdependencies between the features

df['ts'] = pd.to_datetime(df['ts'], unit='ms')
df['registration'] = pd.to_datetime(df['registration'], unit='ms')


print(df.shape)
print(df.dtypes)

(543705, 18)
ts               datetime64[ns]
userId                   object
sessionId                 int64
page                     object
auth                     object
method                   object
status                    int64
level                    object
itemInSession             int64
location                 object
userAgent                object
lastName                 object
firstName                object
registration     datetime64[ns]
gender                   object
artist                   object
song                     object
length                  float64
dtype: object


In [90]:
df_clean = df[df['userId']!=''].drop(columns=['artist','song','length','lastName','firstName'] )
print(df_clean.isnull().sum())

ts               0
userId           0
sessionId        0
page             0
auth             0
method           0
status           0
level            0
itemInSession    0
location         0
userAgent        0
registration     0
gender           0
dtype: int64


In [92]:
from datetime import datetime


df_ = df_clean.sort_values(by=['ts', 'userId','sessionId', 'itemInSession'])
df_.drop_duplicates(inplace=True)
df_['date'] =  df_['ts'].dt.date

label = []
different_days = []

df_label = pd.DataFrame(columns=list(df_.columns)+['chrun', 'days_since_last_session'])

for id in df_['userId'].unique() :
  # print(f'Start id {id} Columns {df_label.columns}')
  temp = df_[df_['userId']==id].copy()
  # print(f'Start temp Columns {temp.columns}')

  label = []
  label.append(0)

  different_days = []
  different_days.append(0)

  # print(temp.head())
  for i in range(1,len(temp)):
    current_level = temp.iloc[i]['level']
    previous_level = temp.iloc[(i-1)]['level']
    if (previous_level == 'paid') & (current_level == 'free') :
      label.append(1)
    else :
      label.append(0)

    current_date = temp.iloc[i]['date']
    previous_date = temp.iloc[(i-1)]['date']
    diff = current_date - previous_date

    different_days.append(diff.days)

  temp['chrun'] = label
  temp['days_since_last_session'] = different_days
  #
  df_label = pd.concat([df_label, temp], ignore_index=True)
  # print(f'Finish id {id} Columns {df_label.columns}')
    # break
  # break




/tmp/ipython-input-3967356636.py:42: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



In [97]:
# Extract features
df_label['year'] = df_label['ts'].dt.year
df_label['month'] = df_label['ts'].dt.month
df_label['day'] = df_label['ts'].dt.day
df_label['hour'] = df_label['ts'].dt.hour
df_label['minute'] = df_label['ts'].dt.minute
df_label['second'] = df_label['ts'].dt.second

# Optionally, drop the original datetime column if not needed
df_label = df_label.drop(columns=['ts'])
df_label

,userId,sessionId,page,auth,method,status,level,itemInSession,location,userAgent,...,gender,date,chrun,days_since_last_session,year,month,day,hour,minute,second
0,293,292,NextSong,Logged In,PUT,200,free,20,"Corpus Christi, TX","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",...,M,2018-10-01,0,0,2018,10,1,0,0,11
1,293,292,NextSong,Logged In,PUT,200,free,21,"Corpus Christi, TX","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",...,M,2018-10-01,0,0,2018,10,1,0,10,8
2,293,292,NextSong,Logged In,PUT,200,free,22,"Corpus Christi, TX","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",...,M,2018-10-01,0,0,2018,10,1,0,13,8
3,293,292,Thumbs Up,Logged In,PUT,307,free,23,"Corpus Christi, TX","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",...,M,2018-10-01,0,0,2018,10,1,0,13,9
4,293,292,NextSong,Logged In,PUT,200,free,24,"Corpus Christi, TX","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",...,M,2018-10-01,0,0,2018,10,1,0,17,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528000,3,369,NextSong,Logged In,PUT,200,free,20,"Bozeman, MT","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",...,M,2018-11-28,0,0,2018,11,28,10,31,56
528001,3,369,NextSong,Logged In,PUT,200,free,21,"Bozeman, MT","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",...,M,2018-11-28,0,0,2018,11,28,10,35,36
528002,3,369,NextSong,Logged In,PUT,200,free,22,"Bozeman, MT","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",...,M,2018-11-28,0,0,2018,11,28,10,38,31
528003,3,369,NextSong,Logged In,PUT,200,free,23,"Bozeman, MT","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",...,M,2018-11-28,0,0,2018,11,28,10,41,56


In [98]:
# Extract features
df_label['reg_year'] = df_label['registration'].dt.year
df_label['reg_month'] = df_label['registration'].dt.month
df_label['reg_day'] = df_label['registration'].dt.day
df_label['reg_hour'] = df_label['registration'].dt.hour
df_label['reg_minute'] = df_label['registration'].dt.minute
df_label['reg_second'] = df_label['registration'].dt.second

# Optionally, drop the original datetime column if not needed
df_label = df_label.drop(columns=['registration'])
df_label

,userId,sessionId,page,auth,method,status,level,itemInSession,location,userAgent,...,day,hour,minute,second,reg_year,reg_month,reg_day,reg_hour,reg_minute,reg_second
0,293,292,NextSong,Logged In,PUT,200,free,20,"Corpus Christi, TX","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",...,1,0,0,11,2018,7,20,5,11,47
1,293,292,NextSong,Logged In,PUT,200,free,21,"Corpus Christi, TX","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",...,1,0,10,8,2018,7,20,5,11,47
2,293,292,NextSong,Logged In,PUT,200,free,22,"Corpus Christi, TX","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",...,1,0,13,8,2018,7,20,5,11,47
3,293,292,Thumbs Up,Logged In,PUT,307,free,23,"Corpus Christi, TX","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",...,1,0,13,9,2018,7,20,5,11,47
4,293,292,NextSong,Logged In,PUT,200,free,24,"Corpus Christi, TX","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",...,1,0,17,36,2018,7,20,5,11,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528000,3,369,NextSong,Logged In,PUT,200,free,20,"Bozeman, MT","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",...,28,10,31,56,2018,8,10,7,29,51
528001,3,369,NextSong,Logged In,PUT,200,free,21,"Bozeman, MT","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",...,28,10,35,36,2018,8,10,7,29,51
528002,3,369,NextSong,Logged In,PUT,200,free,22,"Bozeman, MT","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",...,28,10,38,31,2018,8,10,7,29,51
528003,3,369,NextSong,Logged In,PUT,200,free,23,"Bozeman, MT","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",...,28,10,41,56,2018,8,10,7,29,51


In [99]:
cols_to_convert = ['userId', 'sessionId', 'status', 'itemInSession', 'chrun']  # list of object columns

df_label[cols_to_convert] = df_label[cols_to_convert].astype(int)

In [100]:
cat_cols = ['level', 'page', 'method', 'auth', 'gender']
encoder = OneHotEncoder(sparse_output=False)
encoded = encoder.fit_transform(df_label[cat_cols])
encoded = encoded.astype(int)

encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(cat_cols))

df_model = pd.concat([df_label, encoded_df], axis=1)
df_model = df_model.drop(columns=cat_cols)

In [101]:
X_minority = df_model[df_model['chrun'] == 1]
X_majority = df_model[df_model['chrun'] == 0]

# Duplicate minority events
X_minority_upsampled = X_minority.sample(len(X_majority), replace=True, random_state=42)

df_balanced = pd.concat([X_majority, X_minority_upsampled])

In [102]:
df_balanced.sort_values(by=['year','month','day','hour','minute','second'], inplace=True)

In [103]:
fig = go.Figure(go.Pie(labels=['staied in same plan','changed to free'], values=[(df_balanced['chrun']==0).sum(), (df_balanced['chrun']==1).sum()]))
fig.show()

In [111]:
df_balanced['days_since_last_session'] = df_balanced['days_since_last_session'].astype(int)
# df_balanced = df_balanced.drop(columns='date')

In [112]:
split_index = int(df_balanced.shape[0] * 0.7)

train = df_balanced.iloc[:split_index]
test  = df_balanced.iloc[split_index:]

X_train = train.drop(columns=['chrun', 'location', 'userAgent'])  # returns a new DataFrame
y_train = train['chrun']

X_test = test.drop(columns=['chrun', 'location', 'userAgent'])  # returns a new DataFrame
y_test = test['chrun']

In [113]:
clf = LogisticRegression(class_weight='balanced', max_iter=1000)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



              precision    recall  f1-score   support

           0       1.00      0.98      0.99    163264
           1       0.98      1.00      0.99    153469

    accuracy                           0.99    316733
   macro avg       0.99      0.99      0.99    316733
weighted avg       0.99      0.99      0.99    316733



In [114]:
import joblib

joblib.dump(clf, 'churn_model.pkl')

['churn_model.pkl']